# Evaluation

# Inhouds opgaven Evaluation

## 1. Libraries
## 2. Data Inladen
## 3. Retrieve models
## 4. Functions get prediction
## 5. Example
## 6. Conclusie

# 1. Libraries

Hier importeren we de benodigde libraries voor de evaluatie.

In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


# 2. Retrieve models

Het terug halen van de verschilende modelen en encoders

In [27]:
%%capture
%run 4.Modeling.ipynb

In [28]:
# model_list [0] == liniar regesion model
# model_list[1] == randomforest
# model_list[2] == target encoder
model_list = return_models() 


# 3. Data Inladen

De validatieset inladen.

In [29]:
table = pd.read_csv('Validatieset_2020_2021_CSV.csv', sep=';')

# 4. Functions get prediction

Voor het evalueren hebben we hier een aantal functies voor het toepassen van de modellen en de encoders. get_prediction_forest heeft een model en de data set nodig en loopt de gegeven data set af en voegt de predicties toe aan een lijst en retourneert die.

In [30]:
def get_prediction_forest(model_list, table):
    prediction_list = []
    for item in table.values:
        prediction_list.append(model_list[1].predict([item]))
        # Algemene RSME =55 minuten,  r2=0.79  ( random forest )
    return prediction_list

get_prediction_linreg heeft een model en de data set nodig en loopt de gegeven data set af en voegt de predicties toe aan een lijst en retourneert die.

In [31]:
def get_prediction_linreg(model_list, table):
    prediction_list = []
    for item in table.values:
        prediction_list.append(model_list[0].predict([item]))
        # Algemene RSME =108 minuten,  r2=0.21  ( random linreg )
    return prediction_list

change_target_encoder heeft de encoder en de tabel nog en transformeert de gehele kolom naar de bij behorende encoded categorie mean.

In [32]:
def change_target_encoder(encoder, table):
    table['stm_equipm_soort_mld_looe_mean'] = encoder.transform(table['stm_equipm_soort_mld'].astype('category'))
    return table

Voor de modellen is de duur van ter plaatsen nodig dit wordt hier onder berekend met de datatime objecten in de kolommen.

In [33]:
def change_type_create_duration(table):
    table['stm_aanntpl_ddt'] = pd.to_datetime(table['stm_aanntpl_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
    table['stm_aanngeb_ddt'] = pd.to_datetime(table['stm_aanngeb_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
    table['stm_gebeld_ter_plekken_duur'] = (table['stm_aanntpl_ddt'] - table['stm_aanngeb_ddt']).astype('timedelta64[m]')
    table['Functie Herstel Tijd Duur'] = pd.DataFrame()
    return table
    

Hier maken we de functie herstel duur aan van de validatie set

In [41]:
def aanmaken_functie_herstel_duur(table):
    table['stm_sap_meld_ddt'] = pd.to_datetime(table['stm_sap_meld_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
    table['stm_fh_ddt'] = pd.to_datetime(table['stm_fh_ddt'], errors='coerce', dayfirst=False, infer_datetime_format=True)
    table['Functie Herstel Tijd Duur'] = (table['stm_fh_ddt'] - table['stm_sap_meld_ddt']).astype('timedelta64[m]')
    return table
    

# 5. Example

Er moeten kolomen aan gemaakt worden zo als duur tussen tijd en target encoding.

In [34]:
table = change_type_create_duration(table)
table = change_target_encoder(model_list[2], table)
table = aanmaken_functie_herstel_duur(table)
# hier boven staan de funcite, mee geven wordt op moment validatie zet en de model[2] encoder
table[['stm_progfh_in_duur', 'stm_equipm_soort_mld_looe_mean', 'stm_gebeld_ter_plekken_duur']]

,stm_progfh_in_duur,stm_equipm_soort_mld_looe_mean,stm_gebeld_ter_plekken_duur
0,30,134.605802,40.0
1,77,102.943865,56.0
2,170,152.680873,16.0


Er worden hier de verschillende predicties gemaakt met de modellen en terug gestopt in de table

In [45]:
table['Linear Prediction'] = pd.DataFrame(get_prediction_linreg(model_list, table[['stm_progfh_in_duur', 'stm_equipm_soort_mld_looe_mean', 'stm_gebeld_ter_plekken_duur']]))
table['Forest Prediction'] = pd.DataFrame(get_prediction_forest(model_list, table[['stm_progfh_in_duur', 'stm_equipm_soort_mld_looe_mean', 'stm_gebeld_ter_plekken_duur']]))

Hier onder zijn de verschilde predicties gegeven met de prognose duur en de echte functie hersteltijd. Er is te zien dat de predicties ver af liggen van de geven functie herstel tijd duur en de prognose duur.

In [47]:
table[['stm_progfh_in_duur', 'Forest Prediction', 'Linear Prediction', 'Functie Herstel Tijd Duur']]

,stm_progfh_in_duur,Forest Prediction,Linear Prediction,Functie Herstel Tijd Duur
0,30,118.411482,81.055597,59.0
1,77,161.337087,110.637050,62.0
2,170,181.977543,110.221309,86.0


# 6. Conclusie

We zien uit de validatie zet dat de prognose dichter in de buurt komt van de lineair regressie model en de random forest. Gebaseerd op de gegeven data set en de features die onderzocht zijn is er geen goed model te maken die een predictie geeft die beter is dan de prognose zelf. Er zou meer onderzoek gedaan moeten worden in de features om te kijken of er niet nog meer categorieën zijn die een betere verdeling hebben functie herstel tijd duur.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=df0a20df-15ef-4481-b361-742b03a94b10' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>